In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段5：部署：使用Vertex AI端点开始
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_endpoints.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_endpoints.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_endpoints.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>         
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK来创建和使用“Vertex AI 终端节点”资源来为模型提供服务。Vertex AI 终端节点提供了将服务二进制文件和服务基础设施虚拟化的能力。

### 目标

在本教程中，您将学习如何使用`Vertex AI Endpoint`资源。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI Endpoints`
- `Vertex AI Models`

执行的步骤包括：

- 创建一个`Endpoint`资源。
- 列出所有的`Endpoint`资源。
- 根据查询过滤器列出`Endpoint`资源。
- 配置一个`Model`资源的serving二进制文件，用于部署到一个`Endpoint`资源。
- 将单个`Model`资源部署到一个`Endpoint`资源。
- 获取已部署的`Model`资源的部署设置。
- 配置自动缩放。
- 将多个`Model`资源部署到一个`Endpoint`资源，并配置流量分担。
- 动态更改`Endpoint`资源的流量分担。
- 从一个`Endpoint`资源中取消部署一个单个`Model`资源。
- 从一个`Endpoint`资源中取消部署所有的`Model`资源。
- 删除一个`Endpoint`资源。
- 在流水线中：创建一个`Endpoint`资源并将一个现有的`Model`资源部署到`Endpoint`资源。
- 在流水线中：将一个现有的`Model`资源部署到一个现有的`Endpoint`资源。

数据集

本教程使用了来自TensorFlow Hub的预训练图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多有关[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 成本

此教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[云存储
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

### 重启内核

在安装了额外的包之后，您需要重启笔记本内核，以便它能够找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果可以的话，请确保在 GPU 运行时下运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置 Google Cloud 项目

**以下步骤是必需的，无论您使用何种笔记本环境。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 $300 的免费信用用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 使用正确的项目来运行此笔记本中的所有命令。

**注意**: Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并插值以 `$` 为前缀的 Python 变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于整个笔记本的操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太: `asia-east1`

您可能不应该使用多区域存储桶进行Vertex AI训练。并非所有区域均为所有Vertex AI服务提供支持。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在直播教程会话中，您可能会使用共享测试账号或项目。为了避免在创建的资源中用户之间发生命名冲突，您必须为每个实例会话创建一个时间戳，并将时间戳追加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经通过身份验证。

**如果您正在使用Colab**，请运行下面的单元格，并在提示时按照说明通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中，输入一个名称，然后点击**创建**。

在**授予此服务帐户访问权限的项目**部分，点击角色下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“Storage对象管理员”，然后选择**Storage对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储存储桶

**无论您使用的是哪个笔记本环境，都需要执行以下步骤。**

当您初始化用于 Python 的 Vertex AI SDK 时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源在会话之间保留的位置。

请在下方设置您的云存储存储桶的名称。存储桶的名称必须在全局范围内在所有谷歌云项目中是唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有在您的存储桶不存在时才运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持 GPU 的容器映像，并为虚拟机实例（VM）分配的 GPU 数量。例如，要使用一个GPU容器映像，并为每个VM分配4个 Nvidia Telsa K80 GPU，您需要指定:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用一个容器映像在CPU上运行。

了解更多关于[您区域的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*: 2.3 之前的 TF 发布版本因 GPU 支持而无法加载此教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已修复。这是由生成在服务功能中的静态图运算导致的。如果您在自己的自定义模型中遇到此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 设置变量`DEPLOY_COMPUTE`以配置用于预测的虚拟机的计算资源。
- `机器类型`
   - `n1-standard`：每个虚拟CPU 3.75GB内存。
   - `n1-highmem`：每个虚拟CPU 6.5GB内存。
   - `n1-highcpu`：每个虚拟CPU 0.9GB内存。
- `vCPUs`：\ [2、4、8、16、32、64、96 \]的数量。

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从TensorFlow Hub获取预训练模型

为了演示目的，本教程使用了来自TensorFlow Hub（TFHub）的预训练模型，然后将其上传到“Vertex AI Model”资源中。一旦您拥有了“Vertex AI Model”资源，就可以将模型部署到“Vertex AI Endpoint”资源中。

### 下载预训练模型

首先，您需要从TensorFlow Hub下载预训练模型。该模型将作为一个TF.Keras层进行下载。在本例中，为了完成模型，您将使用下载的TFHub模型创建一个`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型工件

在这一点上，模型已经加载到内存中。接下来，您需要将模型工件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

### 将TensorFlow Hub模型上传到“Vertex AI Model”资源

最后，您将来自TFHub模型的模型文件上传到“Vertex AI Model”资源。

*注意：*当您将模型文件上传到“Vertex Model”资源时，您需要指定相应的部署容器镜像。

In [ ]:
model = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

## 创建 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法创建一个 `Endpoint` 资源。至少，您需要为端点指定显示名称。可选地，您可以指定项目和位置（区域）；否则，设置会继承您在使用 `init()` 方法初始化 Vertex AI SDK 时设置的值。

在这个例子中，指定了以下参数：

- `display_name`：`Endpoint` 资源的可读名称。
- `project`：您的项目 ID。
- `location`：您的区域。
- `labels`：（可选）以键值对形式定义的用于 `Endpoint` 的用户自定义元数据。

该方法返回一个 `Endpoint` 对象。

了解更多关于 [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

获取“Endpoint”资源的详细信息

您可以使用属性“gca_resource”获取“Endpoint”对象的基础详细信息。

In [ ]:
print(endpoint.gca_resource)

### 列出`端点`

方法 `Endpoint.list()` 将返回您的项目所有`端点`资源的列表。

In [ ]:
endpoints = aip.Endpoint.list()
print(len(endpoints))

### 使用过滤器列出`Endpoints`

您可以使用参数`filter`来缩小从`list()`方法返回的`Endpoint`资源。 参数的格式如下：

    filter='<endpoint-property>=<value>[AND <endpoint-property>=<value>, ...]'
    
在这个例子中，您可以根据单个`Endpoint`属性`display_name`进行过滤。

如果`list()`方法返回多个`Endpoints`，您可以使用参数`order_by`对它们进行排序。 参数的格式如下：

    order_by='<endpoint-property?'
    
在这个例子中，您通过`Endpoint`属性`create_time`对返回的列表进行排序。

In [ ]:
aip.Endpoint.list(filter="display_name=example_" + TIMESTAMP, order_by="create_time")

## 将`Model`资源部署到`Endpoint`资源。

您可以将一个或多个`Vertex AI Model`资源实例部署到同一个端点上。每个部署的`Vertex AI Model`资源都将拥有自己的用于服务二进制文件的部署容器。

*注意：* 在这个例子中，您在上传模型文件到`Vertex AI Model`资源的前一步中指定了TFHub模型的部署容器。

### 部署单个`Endpoint`资源

在下一个例子中，您将一个`Vertex AI Model`资源部署到一个`Vertex AI Endpoint`资源中。`Vertex AI Model`资源已经为它定义了部署容器镜像。要部署，您需要指定以下额外的配置设置：

- 机器类型。
- GPU的类型和数量（如果有）。
- 静态、手动或自动扩展的VM实例。

在这个例子中，您以最少的指定参数部署模型，如下：

- `model`：`Model`资源。
- `deployed_model_displayed_name`：部署模型实例的人类可读的名称。
- `machine_type`：每个VM实例的机器类型。

由于资源预设要求，这可能需要几分钟时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

获取部署模型的信息

您可以从`Endpoint`资源配置数据`gca_resource.deployed_models`中获取部署模型的部署设置。在这个例子中，只有一个模型被部署 -- 因此引用了下标`[0]`。

In [ ]:
print(endpoint.gca_resource.deployed_models[0])

### 从 `Endpoint` 资源中撤销 `Model` 资源

当一个 `Model` 资源被部署到一个 `Endpoint` 资源时，已部署的 `Model` 资源实例会被分配一个 ID -- 通常称为已部署的模型 ID。

您可以使用 `undeploy()` 方法撤销特定的 `Model` 资源实例，具有以下参数：

- `deployed_model_id`：已部署模型分配的ID。

In [ ]:
deployed_model_id = endpoint.gca_resource.deployed_models[0].id
print(deployed_model_id)

endpoint.undeploy(deployed_model_id)

## 配置 GPU 资源和扩展规模

在下一个示例中，您还可以为硬件加速器（GPU）和 VM 实例数量配置部署的模型，具有以下附加参数：

- `accelerator_type`：硬件加速器（GPU）的类型。
- `accelerator_count`：每个 VM 实例的硬件加速器数量。
- `min_replica_count`：自动扩展的最小 VM 实例数。*注意：* 必须至少为一。
- `max_replica_count`：自动扩展的最大 VM 实例数。

`Vertex AI Endpoints` 支持以下类型的扩展规模。

- 单个实例：最小和最大复制计数均设置为 1。
- 手动扩展：最小和最大复制计数设置为大于 1 的相同值。在这种情况下，(最大) VM 实例数量在启动时配置并保持恒定。
- 自动扩展：最大复制计数大于最小复制计数。在启动时，将配置最小数量的 VM 实例，并根据负载情况动态增加到最大复制计数，然后再减少至最小复制计数。

In [ ]:
MIN_NODES = 1
MAX_NODES = 2


response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

deployed_model_id = endpoint.gca_resource.deployed_models[0].id

### 将多个 `Model` 资源部署到一个 `Endpoint` 资源

一个 `Endpoint` 资源可以有多个部署的模型。当向一个 `Endpoint` 发送预测请求时，它将根据负载均衡路由到其中一个部署的模型。在同一个 `Endpoint` 资源上部署的所有模型必须是同质的 -- 即，具有相同的输入向量和输出。

当您将多个 `Model` 资源部署到一个 `Endpoint` 时，你需要指定如何在不同部署的 `Model` 资源之间分配预测流量。一个常见的用法是进行 `金丝雀发布`，其中有一个新的生产版本的模型，并逐步将其推出 -- 观察对新模型没有负面影响。

参数 `traffic_split` 的格式如下：

{ "0": 百分比, deploy_model_id: 百分比, ... }

键 "0" 是要部署的模型。在这个例子中，它将占到 10%。每个后续的键/值对都指的是现有部署模型，其中键是部署模型的ID，值是该模型的新百分比。所有值必须加起来等于 100（100%）。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
    traffic_split={"0": 10, deployed_model_id: 90},
)

print(endpoint.gca_resource.deployed_models)

调整流量分流

目前，在SDK中不支持重新配置流量分流（但将来会支持）。您可以使用GAPIC API接口动态重新配置流量分流。动态重新配置流量分流的一个示例是逐步推出金丝雀版本。

要实现这一点，您需要：

1. 使用调用`gapic.EndpointServiceClient()`创建客户端接口
2. 使用方法`get_endpoint()`获取相应端点的GAPIC对象
3. 更新内存中的GAPIC端点对象中的流量分流
4. 使用方法`update_endpoint()`来动态更新负载均衡器上`Endpoint`资源的流量分流设置。

#### 创建客户端接口

首先，创建GAPIC客户端接口。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_endpoint_client():
    client = aip.gapic.EndpointServiceClient(client_options=client_options)
    return client


clients = {}
clients["endpoint"] = create_endpoint_client()

traffic_split = endpoint.traffic_split
print(traffic_split)
new_traffic_split = {}
for key, value in traffic_split.items():
    if value == 90:
        value = 80
    else:
        value = 20
    new_traffic_split[key] = value
print(new_traffic_split)

更新流量分配

接下来，您可以在内存中的GAPIC端点上设置新的流量分配，然后使用`update_endpoint()`方法将其推送到`Endpoint`资源上的负载均衡器。

In [ ]:
from google.protobuf.field_mask_pb2 import FieldMask

gapic_endpoint = clients["endpoint"].get_endpoint(name=endpoint.resource_name)

gapic_endpoint.traffic_split = new_traffic_split
gapic_endpoint.deployed_models = []

clients["endpoint"].update_endpoint(
    endpoint=gapic_endpoint, update_mask=FieldMask(paths=["traffic_split"])
)

# refetch the endpoint
gapic_endpoint = clients["endpoint"].get_endpoint(name=endpoint.resource_name)
print(gapic_endpoint.traffic_split)

### 从一个`Endpoint`资源中撤销一个单独的`Model`资源。

接下来，您可以从一个具有多个部署的`Model`实例的`Endpoint`资源中撤销一个`Model`实例。

In [ ]:
model_0 = endpoint.gca_resource.deployed_models[0].id
model_1 = endpoint.gca_resource.deployed_models[1].id

endpoint.undeploy(model_0)
print(endpoint.gca_resource.deployed_models)

最后，您可以使用 `undeploy_all()` 方法从 `Endpoint` 资源中取消部署所有 `Model` 实例。

In [ ]:
endpoint.undeploy_all()

### 删除一个`Endpoint`资源

如果`Endpoint`资源没有部署的模型，可以使用`delete()`方法删除该资源。

In [ ]:
endpoint.delete()

## `终端点`作为`Vertex AI Pipeline`的一部分

接下来的部分演示了如何在`Vertex AI Pipeline`中使用`终端点`资源。

### 在管道中创建`终端点`

在这个管道中，您将创建一个`终端点`资源，然后将一个`模型`资源部署到`终端点`资源上。要部署的`模型`资源是您之前作为`模型`资源导入的现有TFHub模型。操作步骤如下：

- 对于管道参数，传递现有`模型`资源的资源名称。
- 使用`GetVertexModelOp()`组件为模型创建一个`VertexModel`管道工件。
- 创建一个`终端点`资源。
- 使用`VertexModel`管道工件，将`模型`资源部署到`终端点`资源上。

In [ ]:
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import Artifact, Output, component

PIPELINE_ROOT = "{}/pipeline_root/endpoint_example".format(BUCKET_URI)


@dsl.pipeline(
    name="create-endpoint-deploy-model",
    description="create an endpoint and deploy a model",
)
def pipeline(
    display_name: str,
    resource_name: str,
    project: str = PROJECT_ID,
    region: str = REGION,
):
    from google_cloud_pipeline_components.experimental.evaluation import \
        GetVertexModelOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)

    model = GetVertexModelOp(model_resource_name=resource_name)

    endpoint_op = EndpointCreateOp(
        project=project,
        location=region,
        display_name=display_name,
    )

    _ = ModelDeployOp(
        model=model.outputs["model"],
        endpoint=endpoint_op.outputs["endpoint"],
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_machine_type=DEPLOY_COMPUTE,
    )


try:
    compiler.Compiler().compile(
        pipeline_func=pipeline, package_path="create_endpoint_and_deploy_model.json"
    )
except Exception as e:
    print(e)

### 执行管道

接下来执行管道。管道接受以下参数，这些参数作为字典`parameter_values`传递：

- `display_name`：生成的Vertex AI资源的显示名称。
- `resource_name`：现有`Model`资源的资源名称。
- `project`：项目ID。
- `region`：地区。

In [ ]:
try:
    pipeline = aip.PipelineJob(
        display_name="create-endpoint-deploy-pipeline",
        template_path="create_endpoint_and_deploy_model.json",
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
            "display_name": "create_endpoint_and_deploy_model_" + TIMESTAMP,
            "resource_name": model.resource_name,
            "project": PROJECT_ID,
            "region": REGION,
        },
        enable_caching=False,
    )

    pipeline.run()

    ! rm -f create_endpoint_and_deploy_model.json
except Exception as e:
    print(e)

查看管道结果

最后，您将查看管道中每个任务的工件输出。

In [ ]:
import json


def print_pipeline_output(job, output_task_name):
    PROJECT_NUMBER = job.gca_resource.name.split("/")[1]
    print(PROJECT_NUMBER)

    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVAL_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            return EXECUTE_OUTPUT
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            return GCP_RESOURCES
        elif tf.io.gfile.exists(EVAL_METRICS):
            ! gsutil cat $EVAL_METRICS
            return EVAL_METRICS

    return None


try:
    print("endpoint-create")
    artifacts = print_pipeline_output(pipeline, "endpoint-create")
    print("\n\n")
    output = !gsutil cat $artifacts
    output = json.loads(output[0])
    endpoint_id = output["artifacts"]["endpoint"]["artifacts"][0]["metadata"][
        "resourceName"
    ]
    print("model-deploy")
    artifacts = print_pipeline_output(pipeline, "model-deploy")
    print("\n\n")
except Exception as e:
    print(e)

清理管道资源

删除为此示例创建的所有`Vertex AI`资源，除了在下一个示例中使用的`Endpoint`资源。

In [ ]:
try:
    endpoint = aip.Endpoint(endpoint_id)
    endpoint.undeploy_all()

    pipeline.delete()
except Exception as e:
    print(e)

### 将现有的`Endpoint`传递到管道中

在这个管道中，您将一个`Model`资源部署到`Endpoint`资源中。要部署的`Model`资源是您之前导入为`Model`资源的现有TFHub模型。`Endpoint`资源是先前管道示例中的现有`Endpoint`。步骤如下：

- 对于管道参数，请传递现有`Model`和`Endpoint`资源的资源名称和资源URI。
- 使用`importer_node()`组件为该模型创建一个`VertexModel`管道工件。
- 使用`GetVertexModelOp()`组件为该模型创建一个`VertexModel`管道工件。
- 使用`VertexModel`和`VertexEndpoint`管道工件，将`Model`资源部署到`Endpoint`资源中。

*注意:* 该示例目前由内部问题阻止：b/219835305

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/endpoint_example_2".format(BUCKET_URI)


# (WORKAROUND  b/219835305)
@component(
    base_image="python:3.9",
    packages_to_install=["google-cloud-aiplatform"],
)
def return_unmanaged_endpoint(resource_name: str, endpoint: Output[Artifact]):

    endpoint.metadata["resourceName"] = resource_name


@dsl.pipeline(
    name="deploy-model-existing-endpoint",
    description="deploy a model to an existing endpoint",
)
def pipeline(
    display_name: str,
    model_resource_name: str,
    endpoint_resource_uri: str,
    endpoint_resource_name: str,
    project: str = PROJECT_ID,
    region: str = REGION,
):
    from google_cloud_pipeline_components.experimental.evaluation import \
        GetVertexModelOp
    from google_cloud_pipeline_components.v1.endpoint import ModelDeployOp

    # Desired sequence: blocked by b/219835305
    """
    from kfp.v2.components import importer_node
    from google_cloud_pipeline_components.types import artifact_types
    """

    model = GetVertexModelOp(model_resource_name=model_resource_name)

    # Desired sequence: blocked by b/219835305
    """
    endpoint = importer_node.importer(
        artifact_uri=endpoint_resource_uri,
        artifact_class=artifact_types.VertexEndpoint,
        metadata={"resourceName": endpoint_resource_name},
    )
    """

    # (WORKAROUND  b/219835305)
    endpoint = return_unmanaged_endpoint(resource_name=endpoint_resource_name)

    _ = ModelDeployOp(
        model=model.outputs["model"],
        endpoint=endpoint.outputs["endpoint"],
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        dedicated_resources_machine_type=DEPLOY_COMPUTE,
    )


try:
    compiler.Compiler().compile(
        pipeline_func=pipeline, package_path="deploy_model_existing_endpoint.json"
    )
except Exception as e:
    print(e)

### 执行流水线

接下来，您执行流水线。流水线接受以下参数，这些参数作为字典`parameter_values`传递：

- `display_name`：生成的Vertex AI资源的显示名称。
- `model_resource_name`：现有`Model`资源的资源名称。
- `endpoint_resource_name`：现有`Endpoint`资源的资源名称。
- `endpoint_resource_uri`：现有`Endpoint`资源的资源URI。
- `project`：项目ID。
- `region`：地区。

In [ ]:
try:
    pipeline = aip.PipelineJob(
        display_name="deploy-model-existing-endpoint",
        template_path="deploy_model_existing_endpoint.json",
        pipeline_root=PIPELINE_ROOT,
        parameter_values={
            "display_name": "deploy_model_existing_endpoint_" + TIMESTAMP,
            "model_resource_name": model.resource_name,
            "endpoint_resource_name": endpoint.resource_name,
            "endpoint_resource_uri": "https://us-central1-aiplatform.googleapis.com/v1/"
            + endpoint.resource_name,
            "project": PROJECT_ID,
            "region": REGION,
        },
    )

    pipeline.run()

    ! rm -f deploy_model_existing_endpoint.json
except Exception as e:
    print(e)

查看管道结果

最后，您将查看管道中每个任务的工件输出。

In [ ]:
try:
    print("model-deploy")
    artifacts = print_pipeline_output(pipeline, "model-deploy")
    print("\n\n")
except Exception as e:
    print(e)

清理管道资源

删除为此示例创建的所有`Vertex AI`资源。

In [ ]:
try:
    endpoint.undeploy_all()
    endpoint.delete()
    pipeline.delete()
except Exception as e:
    print(e)

## 清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}